## Import

In [5]:
 %matplotlib inline

import random
import torch
from torch import nn, optim
import math
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from matplotlib.pyplot import axhline, axvline, grid, style
import seaborn
seaborn.set(style='ticks')
style.use('seaborn')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

px = 1/plt.rcParams['figure.dpi']  # inches to pixels

## Data

In [6]:
X = torch.FloatTensor(30, 2).to(device)
y = torch.LongTensor(30).to(device)

a = torch.randint( 30, 40, (10, ))
b = torch.randint( 60, 90, (10, ))
X[:10, 0] = a
X[10:20, 0] = b

a = 50 + torch.rand(10)*30
b = 80 + torch.rand(10)*30
X[:10, 1] = a
X[10:20, 1] = b

a = torch.randint( 10, 20, (10, ))
b = 30 + torch.rand(10)*20
X[20:30, 0] = a
X[20:30, 1] = b

y[:10] = 1
y[10:20] = 0
y[20:30] = 0

## Model and Training Definition

In [7]:
learning_rate = 1e-3
lambda_l2 = 1e-5

model = nn.Sequential(
    nn.Linear(2, 50, bias = True),    
    nn.Sigmoid(),    
    nn.Linear(50, 50, bias = True),
    nn.Sigmoid(),    
    nn.Linear(50, 2, bias = True)
)

#overfitting model
model = nn.Sequential(
    nn.Linear(2, 100, bias = True),    
    nn.Tanh(),    
    nn.Linear(100, 100, bias = True),    
    nn.Tanh(),    
    nn.Linear(100, 100, bias = True),
    nn.Tanh(),    
    nn.Linear(100, 100, bias = True),
    nn.Tanh(),    
    nn.Linear(100, 2, bias = True)
)

model.to(device) 

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2

# Training def
def epoch(epoch):
    model.to(device)
    
    y_pred = model(X)

    loss = criterion(y_pred, y)
    score, predicted = torch.max(y_pred, 1)    
    acc = (y == predicted).sum().float() / len(y)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    return loss


## Training

In [8]:
%matplotlib tk


fig, (ax, axloss) = plt.subplots(ncols=2, figsize=(960*px, 480*px))

ax.set_xlabel('Age')
ax.set_ylabel('Body Weight')
ax.set(xlim=(-1.1, 100.1), ylim=(-1.1, 140.1))

axloss.set_xlabel('Epoch')
axloss.set_ylabel('Loss')
axloss.set(xlim=(0, 1000), ylim=(0, 1))

mesh = np.arange(-1.1, 150.1, 1)
xx, yy = np.meshgrid(mesh, mesh)
data = torch.from_numpy(np.vstack((xx.reshape(-1), yy.reshape(-1))).T).float()
result = np.zeros([220,220])

model.cpu()
with torch.no_grad():
    Z = model(data).detach()
Z = np.argmax(Z, axis=1).reshape(xx.shape)
p = [ax.contourf(xx, yy, Z, cmap=plt.cm.Paired_r, alpha=0.3) ]
pl, = axloss.plot([])

sc = ax.scatter(X.cpu().numpy()[:, 0], X.cpu().numpy()[:, 1], c=y.cpu(), s=100, cmap=plt.cm.Paired_r)

def plot_contour():
    if p[0] != None:
        for tp in p[0].collections:
            tp.remove()
    model.cpu()
    with torch.no_grad():
        Z = model(data).detach()
    Z = np.argmax(Z, axis=1).reshape(xx.shape)
 
    p[0] = ax.contourf(xx, yy, Z, cmap=plt.cm.Paired_r, alpha=0.3)   

    return p[0]

titleLoss = axloss.text(0.5,0.85, "", bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},
                transform=axloss.transAxes, ha="center")

epochs = np.arange(0, 1000)
yloss = np.zeros(1000)

def animate(i):
    loss = epoch(i)
    yloss[i] = loss.item()

    plot_contour()
    pl.set_data(epochs[:i], yloss[:i])
    titleLoss.set_text("Epoch %d / Loss %.3f" % (i, loss.item()))

anim = FuncAnimation(fig, animate, frames = epochs, interval=1, repeat=False, blit = False)
# anim.save('./fig/training_blood_nonlinear1.gif', writer='pillow', fps=30, dpi=100)
plt.show()